Testowane na Google Colab z resnet50_coco_best_v2.1.0.h5

In [0]:
!pip install -q git+https://github.com/fizyr/keras-retinanet

In [0]:
import keras

from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color
from keras_retinanet.utils.gpu import setup_gpu

import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
import time

labels_to_names = {
    0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane',
    5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light',
    10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench',
    14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow',
    20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack',
    25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee',
    30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite',
    34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard',
    37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass',
    41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl',
    46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli',
    51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake',
    56: 'chair', 57: 'couch', 58: 'potted plant', 59: 'bed',
    60: 'dining table', 61: 'toilet', 62: 'tv', 63: 'laptop', 64: 'mouse',
    65: 'remote', 66: 'keyboard', 67: 'cell phone', 68: 'microwave',
    69: 'oven', 70: 'toaster', 71: 'sink', 72: 'refrigerator', 73: 'book',
    74: 'clock', 75: 'vase', 76: 'scissors', 77: 'teddy bear',
    78: 'hair drier', 79: 'toothbrush'}

model = models.load_model('resnet50_coco_best_v2.1.0.h5', backbone_name='resnet50')
setup_gpu(0)

def analyze_image(image, labels):
  global model
  draw = image.copy()
  draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

  image = preprocess_image(image)
  image, scale = resize_image(image)

  start = time.time()
  boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))
  print("Processing time: ", time.time() - start)

  boxes /= scale

  for box, score, label in zip(boxes[0], scores[0], labels[0]):
      if score < 0.5:
          break

      color = label_color(label)

      b = box.astype(int)
      draw_box(draw, b, color=color)

      caption = "{} {:.3f}".format(labels_to_names[label], score)
      draw_caption(draw, b, caption)

  plt.figure(figsize=(15, 15))
  plt.axis('off')
  plt.imshow(draw)
  plt.show()

images = [
          '000000284962.jpg',
          '000000366550.jpg',
          '000000370553.jpg',
          '000000521272.jpg',
          '000000530976.jpg',
          '000000532894.jpg'
]

for image_path in images:
  image = read_image_bgr(image_path)
  analyze_image(image, labels_to_names)

In [0]:
def grab_camera_snapshot(camera_id=0):
    camera = cv2.VideoCapture(camera_id)
    try:
        for i in range(15):
            snapshot_ok, image = camera.read()
        if not snapshot_ok:
            raise "Couldn't take an image"

    finally:
        camera.release()
    return image


try:
  image = grab_camera_snapshot()
  analyze_image(image, labels_to_names)
except:
  print('There was an error when trying to take a picture')